In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Bucketing").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 23:26:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/22 23:26:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


24/02/22 23:26:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [15]:
spark

- Check if bucketing is Enabled

In [4]:
spark.conf.get("spark.sql.sources.bucketing.enabled")

'true'

- create sample data for demo

In [5]:
from pyspark.sql.functions import col, rand

In [6]:
df = spark.range(1, 10000, 1, 10).select(
    col("id").alias("PK"), rand(10).alias("Attribute")
)
df.show()

+---+-------------------+
| PK|          Attribute|
+---+-------------------+
|  1| 0.1709497137955568|
|  2| 0.8051143958005459|
|  3| 0.5775925576589018|
|  4| 0.9476047869880925|
|  5|    0.2093704977577|
|  6|0.36664222617947817|
|  7| 0.8078688178371882|
|  8| 0.7135143433452461|
|  9| 0.7195325566306053|
| 10|0.31335292311175456|
| 11| 0.8062503712025726|
| 12|0.10814914646176654|
| 13| 0.3362232980701172|
| 14| 0.8133304803837667|
| 15|0.47649428738170896|
| 16|  0.524728096293865|
| 17| 0.9701253460019921|
| 18| 0.6232167713919952|
| 19| 0.5089687568245219|
| 20| 0.5467504094508642|
+---+-------------------+
only showing top 20 rows



In [7]:
df.count()

9999

- Create Non-Bucketing table

In [8]:
df.write.format("parquet").saveAsTable("nonbucketedTable")

24/02/22 23:32:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


- create bucketed table

In [9]:
df.write.format("parquet").bucketBy(10, "PK").saveAsTable("bucketedTable")

- Created Bucked and Non-Bucketed Dataframes for Demo

In [10]:
df1 = spark.table("bucketedTable")
df2 = spark.table("bucketedTable")

df3 = spark.table("nonbucketedTable")
df4 = spark.table("nonbucketedTable")

- Broadcast Join by Default if less than 10 MB

In [11]:
df3.join(df4, "PK", "inner").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [PK#92L, Attribute#93, Attribute#99]
   +- BroadcastHashJoin [PK#92L], [PK#98L], Inner, BuildRight, false
      :- Filter isnotnull(PK#92L)
      :  +- FileScan parquet spark_catalog.default.nonbucketedtable[PK#92L,Attribute#93] Batched: true, DataFilters: [isnotnull(PK#92L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/mnt/sda3/iNeuron/Data_Engineering/pySpark/spark-warehouse/nonbuc..., PartitionFilters: [], PushedFilters: [IsNotNull(PK)], ReadSchema: struct<PK:bigint,Attribute:double>
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=122]
         +- Filter isnotnull(PK#98L)
            +- FileScan parquet spark_catalog.default.nonbucketedtable[PK#98L,Attribute#99] Batched: true, DataFilters: [isnotnull(PK#98L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/mnt/sda3/iNeuron/Data_Engineering/pySpark/spark-warehouse/nonbuc..., PartitionFi

- Disable Broadcast join

In [12]:
spark.conf.set("saprk.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.sql.adaptive.enabled", False)

In [13]:
df3.join(df4, "PK", "inner").show()

+----+-------------------+-------------------+
|  PK|          Attribute|          Attribute|
+----+-------------------+-------------------+
|5000|0.18141810315190554|0.18141810315190554|
|5001|  0.626988129685516|  0.626988129685516|
|5002|0.35421803636733495|0.35421803636733495|
|5003|0.21101605231806198|0.21101605231806198|
|5004| 0.8189059402224587| 0.8189059402224587|
|5005| 0.6920078838955664| 0.6920078838955664|
|5006| 0.7664589254286232| 0.7664589254286232|
|5007| 0.5541206475613204| 0.5541206475613204|
|5008| 0.3621346554845799| 0.3621346554845799|
|5009|0.48110035768197124|0.48110035768197124|
|5010|0.18924905548078141|0.18924905548078141|
|5011| 0.4865077798603922| 0.4865077798603922|
|5012| 0.6609186090133667| 0.6609186090133667|
|5013| 0.6393114115238754| 0.6393114115238754|
|5014| 0.6378813264876587| 0.6378813264876587|
|5015| 0.5207868024664773| 0.5207868024664773|
|5016| 0.6128985300949695| 0.6128985300949695|
|5017| 0.1449002210923449| 0.1449002210923449|
|5018|0.04824

- Non-bucketed to Non-bucketed join, Both sides would be shuffled

In [14]:
df3.join(df4, "PK", "inner").explain()

== Physical Plan ==
*(2) Project [PK#92L, Attribute#93, Attribute#125]
+- *(2) BroadcastHashJoin [PK#92L], [PK#124L], Inner, BuildRight, false
   :- *(2) Filter isnotnull(PK#92L)
   :  +- *(2) ColumnarToRow
   :     +- FileScan parquet spark_catalog.default.nonbucketedtable[PK#92L,Attribute#93] Batched: true, DataFilters: [isnotnull(PK#92L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/mnt/sda3/iNeuron/Data_Engineering/pySpark/spark-warehouse/nonbuc..., PartitionFilters: [], PushedFilters: [IsNotNull(PK)], ReadSchema: struct<PK:bigint,Attribute:double>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=228]
      +- *(1) Filter isnotnull(PK#124L)
         +- *(1) ColumnarToRow
            +- FileScan parquet spark_catalog.default.nonbucketedtable[PK#124L,Attribute#125] Batched: true, DataFilters: [isnotnull(PK#124L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/mnt/sda3/iNeuron/Data_Engineering/pySpark/spar

In [16]:
df3.join(df4, "PK", "inner").show()

+----+-------------------+-------------------+
|  PK|          Attribute|          Attribute|
+----+-------------------+-------------------+
|5000|0.18141810315190554|0.18141810315190554|
|5001|  0.626988129685516|  0.626988129685516|
|5002|0.35421803636733495|0.35421803636733495|
|5003|0.21101605231806198|0.21101605231806198|
|5004| 0.8189059402224587| 0.8189059402224587|
|5005| 0.6920078838955664| 0.6920078838955664|
|5006| 0.7664589254286232| 0.7664589254286232|
|5007| 0.5541206475613204| 0.5541206475613204|
|5008| 0.3621346554845799| 0.3621346554845799|
|5009|0.48110035768197124|0.48110035768197124|
|5010|0.18924905548078141|0.18924905548078141|
|5011| 0.4865077798603922| 0.4865077798603922|
|5012| 0.6609186090133667| 0.6609186090133667|
|5013| 0.6393114115238754| 0.6393114115238754|
|5014| 0.6378813264876587| 0.6378813264876587|
|5015| 0.5207868024664773| 0.5207868024664773|
|5016| 0.6128985300949695| 0.6128985300949695|
|5017| 0.1449002210923449| 0.1449002210923449|
|5018|0.04824

- Non-bucketed to Non-bucketed join, One side would be shuffled

In [17]:
df3.join(df1, "PK").explain()

== Physical Plan ==
*(2) Project [PK#92L, Attribute#93, Attribute#87]
+- *(2) BroadcastHashJoin [PK#92L], [PK#86L], Inner, BuildLeft, false
   :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=344]
   :  +- *(1) Filter isnotnull(PK#92L)
   :     +- *(1) ColumnarToRow
   :        +- FileScan parquet spark_catalog.default.nonbucketedtable[PK#92L,Attribute#93] Batched: true, DataFilters: [isnotnull(PK#92L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/mnt/sda3/iNeuron/Data_Engineering/pySpark/spark-warehouse/nonbuc..., PartitionFilters: [], PushedFilters: [IsNotNull(PK)], ReadSchema: struct<PK:bigint,Attribute:double>
   +- *(2) Filter isnotnull(PK#86L)
      +- *(2) ColumnarToRow
         +- FileScan parquet spark_catalog.default.bucketedtable[PK#86L,Attribute#87] Batched: true, Bucketed: false (disabled by query planner), DataFilters: [isnotnull(PK#86L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/mnt/sda3/iN

In [18]:
df3.join(df1, "PK").show()

+----+--------------------+--------------------+
|  PK|           Attribute|           Attribute|
+----+--------------------+--------------------+
|5004|  0.8189059402224587|  0.8189059402224587|
|5005|  0.6920078838955664|  0.6920078838955664|
|5016|  0.6128985300949695|  0.6128985300949695|
|5021| 0.11717753054645008| 0.11717753054645008|
|5030|  0.9748810778061268|  0.9748810778061268|
|5061|  0.7166999415848043|  0.7166999415848043|
|5063|  0.6289109154464138|  0.6289109154464138|
|5076|  0.7869295678081726|  0.7869295678081726|
|5080|  0.5114215285181372|  0.5114215285181372|
|5095| 0.09274336770599478| 0.09274336770599478|
|5127|  0.9905368964232141|  0.9905368964232141|
|5129|  0.5371471706713082|  0.5371471706713082|
|5140| 0.09013460432692155| 0.09013460432692155|
|5150|  0.6534999902211204|  0.6534999902211204|
|5156|  0.9531152736774249|  0.9531152736774249|
|5157|   0.659088997485559|   0.659088997485559|
|5175|0.010991471549378407|0.010991471549378407|
|5187|  0.1946353897

- Non-bucketed to Non-bucketed join, no shuffle at both sides

In [19]:
df.join(df2, "PK").explain()

== Physical Plan ==
*(2) Project [PK#2L, Attribute#3, Attribute#87]
+- *(2) BroadcastHashJoin [PK#2L], [PK#86L], Inner, BuildLeft, false
   :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=462]
   :  +- *(1) Project [id#0L AS PK#2L, rand(10) AS Attribute#3]
   :     +- *(1) Range (1, 10000, step=1, splits=10)
   +- *(2) Filter isnotnull(PK#86L)
      +- *(2) ColumnarToRow
         +- FileScan parquet spark_catalog.default.bucketedtable[PK#86L,Attribute#87] Batched: true, Bucketed: false (disabled by query planner), DataFilters: [isnotnull(PK#86L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/mnt/sda3/iNeuron/Data_Engineering/pySpark/spark-warehouse/bucket..., PartitionFilters: [], PushedFilters: [IsNotNull(PK)], ReadSchema: struct<PK:bigint,Attribute:double>




In [20]:
df.join(df2, "PK").show()

+----+--------------------+--------------------+
|  PK|           Attribute|           Attribute|
+----+--------------------+--------------------+
|5004|  0.8189059402224587|  0.8189059402224587|
|5005|  0.6920078838955664|  0.6920078838955664|
|5016|  0.6128985300949695|  0.6128985300949695|
|5021| 0.11717753054645008| 0.11717753054645008|
|5030|  0.9748810778061268|  0.9748810778061268|
|5061|  0.7166999415848043|  0.7166999415848043|
|5063|  0.6289109154464138|  0.6289109154464138|
|5076|  0.7869295678081726|  0.7869295678081726|
|5080|  0.5114215285181372|  0.5114215285181372|
|5095| 0.09274336770599478| 0.09274336770599478|
|5127|  0.9905368964232141|  0.9905368964232141|
|5129|  0.5371471706713082|  0.5371471706713082|
|5140| 0.09013460432692155| 0.09013460432692155|
|5150|  0.6534999902211204|  0.6534999902211204|
|5156|  0.9531152736774249|  0.9531152736774249|
|5157|   0.659088997485559|   0.659088997485559|
|5175|0.010991471549378407|0.010991471549378407|
|5187|  0.1946353897